In [48]:
schema_ = {
    "properties": {
            "reasoning_2": {
            "description": "Gie a rating between 500 and 10000!",
            "title": "Score",
            "type": "integer",
        },
        "zeasoning": {
            "description": "Why did you give this rating?",
            "title": "Reasoning",
            "type": "string",
        },

        "school_yourself": {
            "description": "Shool yourself. Why is your answer bad?",
            "title": "School Yourself",
            "type": "string",
        },
    },
    "required": ["reasoning_2", "zeasoning", "school_yourself"],
    "propertyOrdering": ["reasoning_2", "zeasoning", "school_yourself"],
    "title": "Test",
    "type": "object",
    "strict": True,
}

In [42]:
from google import genai
from google.genai import types
import base64
from pydantic import BaseModel, Field
from icecream import ic

class Test(BaseModel):
    reasoning: str = Field(description="Why did you give this rating?")
    score: int = Field(
        description="Write a rating of REWE from 500-1000!",
    )
    school_yourself: str = Field(description="Shool yourself. Why is your answer bad?")
#ic(Test.model_json_schema())

In [ ]:
def generate():
    client = genai.Client(
        vertexai=True,
        project="rd-ri-genai-dev-2352",
        #project="rd-smrk-prod518-dev",
        location="us-central1",
    )

    text1 = types.Part.from_text("""How emotional is this sentence: "I am exctatic over my new shoes!""")
    response_schema = Test.model_json_schema
    
    print(response_schema)
    

    model = "gemini-2.0-flash-exp"
    contents = [types.Content(role="user", parts=[text1])]
    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        max_output_tokens=8192,
        response_mime_type="application/json",
        response_schema=schema_,
        response_modalities=["TEXT"],
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(
                category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"
            ),
            types.SafetySetting(
                category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"
            ),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
        ],
    )

    response = client.models.generate_content(
        model=model,
        contents=contents,
        config=generate_content_config,
    )
    return response


response = generate()
print(response.text)

<bound method BaseModel.model_json_schema of <class '__main__.Test'>>


In [47]:
response.text

'{\n"reasoning": "The sentence uses the word \'ecstatic,\' which is a strong positive emotion. The exclamation point further emphasizes the high level of excitement.",\n"reasoning_2": 8500,\n"school_yourself": "I did not provide an actual rating on a scale as requested, so I should make sure to pay closer attention to how the fields are defined"\n}'

In [7]:
class Test(BaseModel):
    reasoning: str = Field(description="Why did you give this rating?")
    score: int = Field(
        description="Write a rating of REWE from 500-1000!",
    )
    school_yourself: str = Field(description="Shool yourself. Why is your answer bad?", ge=500, le=1000)


Test.model_json_schema()

{'properties': {'reasoning': {'description': 'Why did you give this rating?',
   'title': 'Reasoning',
   'type': 'string'},
  'score': {'description': 'Write a rating of REWE from 500-1000!',
   'title': 'Score',
   'type': 'integer'},
  'school_yourself': {'description': 'Shool yourself. Why is your answer bad?',
   'ge': 500,
   'le': 1000,
   'title': 'School Yourself',
   'type': 'string'}},
 'required': ['reasoning', 'score', 'school_yourself'],
 'title': 'Test',
 'type': 'object'}